> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [Foundry IQ 개요](#foundry-iq-개요)
- [AI Search 연결](#ai-search-연결)
- [Knowledge Base 생성 (AI Search Index)](#knowledge-base-생성-ai-search-index)
- [Knowledge Base 생성 (Blob Storage)](#knowledge-base-생성-blob-storage)
- [KnowledgeAgent 통합](#knowledgeagent-통합)

## 🎯 학습 목표

- Foundry IQ의 개념과 장점 이해
- Azure AI Search 리소스 연결 및 구성
- AI Search Index 기반 Knowledge Base 생성
- Blob Storage 기반 Knowledge Base
- Knowledge Base를 에이전트에 통합하는 방법 학습

## ⏱️ 예상 소요 시간

약 40분

## Foundry IQ 개요

### Foundry IQ란?

Foundry IQ는 Microsoft Foundry의 지능형 지식 관리 시스템으로, 다양한 데이터 소스를 통합하여 AI 에이전트에 맥락적 지식을 제공합니다.

### 주요 특징

```
Foundry IQ = Retrieval + Reasoning + Ranking
```

- **Retrieval**: 관련 정보를 효율적으로 검색
- **Reasoning**: 검색된 정보를 이해하고 해석
- **Ranking**: 가장 관련성 높은 정보를 우선순위화

## 환경 설정

Knowledge Base 구축을 위한 Azure 리소스를 설정합니다.

### Azure 환경 변수 및 패키지 로드

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-projects azure-identity azure-search-documents requests

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Project Client 초기화
credential = DefaultAzureCredential()

# 프로젝트 이름 추출 (URL에서 마지막 부분)
# 예: https://foundry-xxx.services.ai.azure.com/api/projects/default-project
# → project_name = "default-project"
import re
match = re.search(r'/projects/([^/]+)$', PROJECT_ENDPOINT)
if match:
    project_name = match.group(1)
else:
    project_name = PROJECT_NAME  # fallback to config

# Foundry URL (프로젝트 부분 제외)
foundry_base_url = PROJECT_ENDPOINT.rsplit('/projects/', 1)[0]

project_client = AIProjectClient(
    endpoint=foundry_base_url,
    credential=credential,
    project_name=project_name
)

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
print(f"✅ Project Client 초기화 완료")
print(f"   Foundry: {foundry_base_url}")
print(f"   Project: {project_name}")

### AI Search 및 Storage 리소스 이름 생성

In [ ]:
# AI Search 및 Storage 리소스 이름 설정
import re

# 고유한 이름 생성 (소문자, 숫자, 하이픈만 허용)
def sanitize_name(name, max_length=24):
    """리소스 이름을 Azure 규칙에 맞게 정리"""
    # 소문자로 변환하고 영숫자만 남김
    clean = re.sub(r'[^a-z0-9]', '', name.lower())
    return clean[:max_length]

# FOUNDRY_NAME 기반으로 고유 이름 생성
base_name = sanitize_name(FOUNDRY_NAME)

SEARCH_NAME = f"{base_name}-search"[:64]  # AI Search는 최대 64자
STORAGE_NAME = sanitize_name(base_name + "store", 24)  # Storage는 최대 24자, 하이픈 불가
SEARCH_INDEX_NAME = "knowledge-index"

print("📌 생성할 리소스 이름:")
print(f"   AI Search: {SEARCH_NAME}")
print(f"   Storage Account: {STORAGE_NAME}")
print(f"   Search Index: {SEARCH_INDEX_NAME}")

# 설정 파일에 저장
config["SEARCH_NAME"] = SEARCH_NAME
config["STORAGE_NAME"] = STORAGE_NAME
config["SEARCH_INDEX_NAME"] = SEARCH_INDEX_NAME

with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n✅ 리소스 이름이 '{config_file}'에 저장되었습니다.")

## Azure AI Search 생성

벡터 검색을 위한 AI Search 리소스를 생성합니다.

### AI Search 리소스 생성 실행

In [ ]:
# AI Search 리소스 생성
!az search service create \
    --name $SEARCH_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku basic

print(f"\n✅ AI Search 생성 완료: {SEARCH_NAME}")

## Blob Storage 생성

문서 파일을 저장할 Blob Storage를 생성합니다.

### Storage Account 및 Container 생성

In [ ]:
# Storage Account 생성
!az storage account create \
    --name $STORAGE_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard_LRS \
    --public-network-access Enabled

print(f"\n✅ Storage Account 생성 완료: {STORAGE_NAME}")

# Storage Container 생성
!az storage container create \
    --name documents \
    --account-name $STORAGE_NAME \
    --auth-mode login

print(f"✅ Container 'documents' 생성 완료!")

## Managed Identity 활성화

AI Search가 Storage와 Foundry에 접근할 수 있도록 Managed Identity를 설정합니다.

In [ ]:
# AI Search의 Managed Identity 활성화
!az search service update \
    --name $SEARCH_NAME \
    --resource-group $RESOURCE_GROUP \
    --identity-type SystemAssigned

print(f"\n✅ AI Search Managed Identity 활성화 완료")

## IAM 권한 설정

Storage Account와 AI Search, Foundry 간의 권한을 설정합니다.

### 사용자 및 리소스 정보 조회

In [ ]:
# 현재 사용자 정보 가져오기
import subprocess
import json

# 현재 로그인한 사용자 Object ID
result = subprocess.run(["az", "ad", "signed-in-user", "show", "--query", "id", "-o", "tsv"], 
                       capture_output=True, text=True)
USER_OBJECT_ID = result.stdout.strip()
print(f"📌 현재 사용자 Object ID: {USER_OBJECT_ID}")

# AI Search의 Principal ID 가져오기
result = subprocess.run([
    "az", "search", "service", "show",
    "--name", SEARCH_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "identity.principalId", "-o", "tsv"
], capture_output=True, text=True)
SEARCH_PRINCIPAL_ID = result.stdout.strip()
print(f"📌 AI Search Principal ID: {SEARCH_PRINCIPAL_ID}")

# Storage Account Resource ID 가져오기
result = subprocess.run([
    "az", "storage", "account", "show",
    "--name", STORAGE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id", "-o", "tsv"
], capture_output=True, text=True)
STORAGE_RESOURCE_ID = result.stdout.strip()
print(f"📌 Storage Resource ID: {STORAGE_RESOURCE_ID}")

### Storage Blob Data Contributor 역할 할당 (사용자 & AI Search MI)

In [ ]:
# 1. Storage Blob Data Contributor - 현재 사용자
print("1️⃣ Storage Blob Data Contributor 역할 할당 (현재 사용자)...")
!az role assignment create \
    --role "Storage Blob Data Contributor" \
    --assignee $USER_OBJECT_ID \
    --scope $STORAGE_RESOURCE_ID

# 2. Storage Blob Data Contributor - AI Search (Managed Identity)
print("\n2️⃣ Storage Blob Data Contributor 역할 할당 (AI Search)...")
!az role assignment create \
    --role "Storage Blob Data Contributor" \
    --assignee $SEARCH_PRINCIPAL_ID \
    --scope $STORAGE_RESOURCE_ID

print("\n✅ Storage IAM 권한 설정 완료!")

### Azure AI Project Manager 역할 할당 (AI Search MI)

In [ ]:
# 3. Foundry 리소스에 Azure AI Project Manager 역할 할당 (AI Search MI)
import subprocess
import os

# Foundry Resource ID 가져오기
result = subprocess.run([
    "az", "cognitiveservices", "account", "show",
    "--name", FOUNDRY_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id", "-o", "tsv"
], capture_output=True, text=True)
FOUNDRY_RESOURCE_ID = result.stdout.strip()

print("3️⃣ Azure AI Project Manager 역할 할당 (AI Search → Foundry)...")
!az role assignment create \
    --role "Azure AI Project Manager" \
    --assignee $SEARCH_PRINCIPAL_ID \
    --scope $FOUNDRY_RESOURCE_ID

print("\n✅ Foundry IAM 권한 설정 완료!")

## 샘플 데이터 업로드

Microsoft에서 제공하는 샘플 데이터를 다운로드하여 Storage Container에 업로드합니다.

In [ ]:
# 샘플 데이터 다운로드 및 업로드
import os
import urllib.request

# 샘플 데이터 URL (Azure Search Sample Data)
sample_files = [
    ("Benefit_Options.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/Benefit_Options.pdf"),
    ("employee_handbook.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/employee_handbook.pdf"),
    ("Northwind_Health_Plus_Benefits_Details.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/Northwind_Health_Plus_Benefits_Details.pdf"),
    ("Northwind_Standard_Benefits_Details.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/Northwind_Standard_Benefits_Details.pdf"),
    ("PerksPlus.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/PerksPlus.pdf"),
    ("role_library.pdf", "https://github.com/Azure-Samples/azure-search-sample-data/raw/main/health-plan/role_library.pdf"),
]

# 임시 디렉토리 생성
os.makedirs("temp_data", exist_ok=True)

# 파일 다운로드
print("📥 샘플 데이터 다운로드 중...")
for filename, url in sample_files:
    filepath = f"temp_data/{filename}"
    if not os.path.exists(filepath):
        print(f"  다운로드: {filename}")
        urllib.request.urlretrieve(url, filepath)
    else:
        print(f"  이미 존재: {filename}")

print("\n✅ 샘플 데이터 다운로드 완료!")

### Storage에 샘플 데이터 업로드

In [ ]:
# Storage Container에 파일 업로드
print("📤 Storage Container에 업로드 중...")

for filename, _ in sample_files:
    filepath = f"temp_data/{filename}"
    print(f"  업로드: {filename}")
    !az storage blob upload \
        --account-name $STORAGE_NAME \
        --container-name documents \
        --file $filepath \
        --name $filename \
        --auth-mode login \
        --overwrite

# 업로드된 파일 확인
print("\n📋 업로드된 파일 목록:")
!az storage blob list \
    --account-name $STORAGE_NAME \
    --container-name documents \
    --auth-mode login \
    --query "[].name" \
    --output table

print("\n✅ 샘플 데이터 업로드 완료!")

# 임시 파일 정리 (선택사항)
# import shutil
# shutil.rmtree("temp_data")

## AI Search Index 생성 (Portal에서 진행)

**⚠️ 중요**: 다음 단계는 Azure Portal에서 수동으로 진행해야 합니다.

### Import Data Wizard 사용 방법:

1. **Azure Portal에서 AI Search 리소스 열기**
   - https://portal.azure.com
   - 생성한 AI Search 서비스 선택

2. **Import data (new) 클릭**
   
3. **Data Source 설정:**
   - Data Source: **Azure Blob Storage**
   - Scenario: **RAG (Retrieval Augmented Generation)**
   - Storage account: `foundry<your-name>`
   - Container: `documents`

4. **Vectorization 설정:**
   - Kind: **Microsoft Foundry**
   - Foundry project: `proj-default`
   - Model deployment: **text-embedding-3-large**
   - Authentication type: **API key**

5. **Semantic Ranker 활성화:**
   - ☑ Enable semantic ranker
   - Schedule: **Once** (초기 인덱싱만)

6. **Review + Create**
   - 설정 확인 후 **Create** 클릭
   - 인덱싱 완료까지 5-10분 소요

완료 후 아래 코드로 인덱스를 확인하세요.

## Python SDK로 AI Search Index 생성

Azure Search Python SDK를 사용하여 벡터 검색이 가능한 인덱스를 생성합니다.

**두 가지 옵션:**
1. **옵션 A (권장)**: 포털의 Import Data Wizard 사용 - 자동으로 데이터 인덱싱
2. **옵션 B**: 아래 Python 코드 사용 - 인덱스만 생성 (데이터는 별도 업로드 필요)

### Azure Search SDK 설치 및 Import

In [ ]:
# 필요한 패키지 설치
%pip install -q azure-search-documents azure-identity

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, 
    SearchField, 
    SearchFieldDataType,
    VectorSearch, 
    VectorSearchProfile, 
    HnswAlgorithmConfiguration,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)
from azure.identity import DefaultAzureCredential

print("✅ 패키지 import 완료")

### AI Search API Key 획득

In [ ]:
# API 키 방식으로 인증 (RBAC 대안)
# 구독 소유자는 API 키를 조회할 수 있습니다
result = subprocess.run([
    "az", "search", "admin-key", "show",
    "--resource-group", RESOURCE_GROUP,
    "--service-name", SEARCH_NAME,
    "--query", "primaryKey", "-o", "tsv"
], capture_output=True, text=True)

SEARCH_API_KEY = result.stdout.strip()

if SEARCH_API_KEY:
    print("✅ AI Search API 키 획득 완료")
    print("💡 API 키 방식을 사용하면 RBAC 역할 없이도 Index를 생성할 수 있습니다.")
else:
    print("⚠️ API 키 획득 실패. DefaultAzureCredential 방식을 사용합니다.")

### Search Index 생성 (벡터 검색 + Semantic Search)

In [ ]:
# AI Search Index 생성
from azure.core.credentials import AzureKeyCredential

# 인증 방식 선택: API 키 우선, 없으면 DefaultAzureCredential
if 'SEARCH_API_KEY' in globals() and SEARCH_API_KEY:
    credential = AzureKeyCredential(SEARCH_API_KEY)
    auth_method = "API 키"
else:
    credential = DefaultAzureCredential()
    auth_method = "DefaultAzureCredential (RBAC)"

search_endpoint = f"https://{SEARCH_NAME}.search.windows.net"

# SearchIndexClient 생성
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=credential
)

print(f"📌 AI Search Endpoint: {search_endpoint}")
print(f"📌 생성할 Index 이름: {SEARCH_INDEX_NAME}")
print(f"🔐 인증 방식: {auth_method}")

# Index 필드 정의
fields = [
    # 고유 식별자
    SearchField(
        name="chunk_id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True
    ),
    # 부모 문서 ID
    SearchField(
        name="parent_id",
        type=SearchFieldDataType.String,
        filterable=True
    ),
    # 문서 제목
    SearchField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
        filterable=True,
        sortable=True
    ),
    # 청크된 텍스트 내용
    SearchField(
        name="chunk",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ko.microsoft"  # 한국어 분석기
    ),
    # 벡터 임베딩 (text-embedding-3-large: 3072 차원)
    SearchField(
        name="text_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,
        vector_search_profile_name="my-vector-profile"
    ),
    # 메타데이터
    SearchField(
        name="category",
        type=SearchFieldDataType.String,
        filterable=True,
        facetable=True
    ),
    SearchField(
        name="sourcepage",
        type=SearchFieldDataType.String,
        filterable=True
    ),
    SearchField(
        name="sourcefile",
        type=SearchFieldDataType.String,
        filterable=True
    )
]

# 벡터 검색 구성
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="my-vector-profile",
            algorithm_configuration_name="my-hnsw-config"
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-hnsw-config",
            parameters={
                "m": 4,  # 그래프 연결 수
                "efConstruction": 400,  # 인덱싱 시 탐색 범위
                "efSearch": 500,  # 검색 시 탐색 범위
                "metric": "cosine"  # 유사도 측정 방식
            }
        )
    ]
)

# Semantic Search 구성 (선택 사항)
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[
            SemanticField(field_name="chunk")
        ]
    )
)

semantic_search = SemanticSearch(
    configurations=[semantic_config]
)

# Index 생성
index = SearchIndex(
    name=SEARCH_INDEX_NAME,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

try:
    # Index 생성 또는 업데이트
    result = index_client.create_or_update_index(index)
    print(f"\n✅ AI Search Index 생성 완료!")
    print(f"   Index 이름: {result.name}")
    print(f"   필드 수: {len(result.fields)}")
    print(f"   벡터 검색: 활성화 (3072 차원)")
    print(f"   Semantic Search: 활성화")
    
    # 설정 파일에 저장
    config["SEARCH_INDEX_NAME"] = SEARCH_INDEX_NAME
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    
except Exception as e:
    print(f"⚠️ Index 생성 실패: {e}")
    print("\n💡 가능한 원인:")
    print("   1. AI Search 리소스가 아직 준비되지 않음 (몇 분 대기)")
    print("   2. 권한 문제:")
    print("      - RBAC 방식: 'Search Index Data Contributor' 역할 필요")
    print("      - 구독 소유자도 Data Plane 권한은 별도로 필요합니다")
    print("   3. 위의 API 키 셀을 먼저 실행하면 RBAC 없이 가능합니다")

### 생성된 Index 정보 확인

In [ ]:
# (선택 사항) 생성된 Index 확인
try:
    # 생성된 Index 가져오기
    created_index = index_client.get_index(SEARCH_INDEX_NAME)
    
    print("📋 Index 상세 정보:")
    print(f"\n필드 목록:")
    for field in created_index.fields:
        field_info = f"  - {field.name} ({field.type})"
        if field.key:
            field_info += " [KEY]"
        if field.searchable:
            field_info += " [검색 가능]"
        if hasattr(field, 'vector_search_dimensions') and field.vector_search_dimensions:
            field_info += f" [벡터: {field.vector_search_dimensions}차원]"
        print(field_info)
    
    print(f"\n벡터 검색 프로필: {len(created_index.vector_search.profiles) if created_index.vector_search else 0}개")
    print(f"Semantic 구성: {len(created_index.semantic_search.configurations) if created_index.semantic_search else 0}개")
    
    # Index 목록 확인
    print("\n📋 전체 Index 목록:")
    indexes = index_client.list_indexes()
    for idx in indexes:
        print(f"  - {idx.name}")
    
except Exception as e:
    print(f"⚠️ Index 정보 조회 실패: {e}")

## 문서 인덱싱 (직접 푸시 방식)

Storage의 PDF 문서를 읽어서 직접 Index에 업로드합니다.
- PDF 파싱 및 텍스트 추출
- 텍스트 청킹 (2000자 단위)
- Foundry API로 임베딩 생성
- AI Search Index에 문서 업로드

### 필수 패키지 설치

In [ ]:
# 필요한 패키지 설치
%pip install -q pypdf azure-storage-blob openai

from azure.storage.blob import BlobServiceClient
from azure.search.documents import SearchClient
from pypdf import PdfReader
from openai import AzureOpenAI
import io
import hashlib
import base64

print("✅ 필요한 패키지 import 완료")

### Storage에서 PDF 다운로드 및 파싱

In [ ]:
# Storage에서 PDF 파일 다운로드
from azure.identity import DefaultAzureCredential

# Blob Service Client 생성
storage_credential = DefaultAzureCredential()
blob_service_client = BlobServiceClient(
    account_url=f"https://{STORAGE_NAME}.blob.core.windows.net",
    credential=storage_credential
)

container_client = blob_service_client.get_container_client("documents")

# 컨테이너의 모든 PDF 파일 나열
print("📥 Storage에서 PDF 파일 다운로드 중...\n")
pdf_files = []

for blob in container_client.list_blobs():
    if blob.name.endswith('.pdf'):
        print(f"  다운로드: {blob.name}")
        blob_client = container_client.get_blob_client(blob.name)
        pdf_data = blob_client.download_blob().readall()
        pdf_files.append({
            'name': blob.name,
            'data': pdf_data
        })

print(f"\n✅ 총 {len(pdf_files)}개 파일 다운로드 완료")

### PDF 텍스트 청킹 및 메타데이터 생성

In [ ]:
# PDF 파싱 및 텍스트 청킹
def chunk_text(text, chunk_size=2000, overlap=200):
    """텍스트를 지정된 크기로 청킹"""
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap  # 오버랩 적용
    
    return chunks

print("📄 PDF 파싱 및 텍스트 청킹 중...\n")
all_chunks = []

for pdf_file in pdf_files:
    try:
        # PDF 읽기
        pdf_reader = PdfReader(io.BytesIO(pdf_file['data']))
        
        # 전체 텍스트 추출
        full_text = ""
        for page_num, page in enumerate(pdf_reader.pages, 1):
            text = page.extract_text()
            if text:
                full_text += text + "\n"
        
        # 텍스트 청킹
        chunks = chunk_text(full_text)
        
        # 메타데이터와 함께 저장
        for chunk_idx, chunk_content in enumerate(chunks):
            chunk_id = hashlib.md5(f"{pdf_file['name']}_{chunk_idx}".encode()).hexdigest()
            
            all_chunks.append({
                'chunk_id': chunk_id,
                'parent_id': pdf_file['name'],
                'title': pdf_file['name'].replace('.pdf', ''),
                'chunk': chunk_content.strip(),
                'sourcefile': pdf_file['name'],
                'sourcepage': f"page_{chunk_idx + 1}",
                'category': 'health-plan'
            })
        
        print(f"  ✓ {pdf_file['name']}: {len(chunks)}개 청크 생성")
        
    except Exception as e:
        print(f"  ⚠️ {pdf_file['name']} 파싱 실패: {e}")

print(f"\n✅ 총 {len(all_chunks)}개 청크 생성 완료")

### Azure OpenAI로 임베딩 생성 및 Index에 업로드

In [ ]:
# Azure OpenAI REST API로 임베딩 생성
print("🔄 임베딩 생성 중...\n")

from azure.identity import DefaultAzureCredential
import requests
import json
import time
import subprocess

# 임베딩 모델 설정
embedding_model = "text-embedding-3-large"
api_version = "2024-02-01"

# Azure OpenAI 엔드포인트 찾기
print("🔍 리소스 그룹에서 Azure OpenAI 계정 찾는 중...")

# Azure CLI로 리소스 그룹의 Cognitive Services 계정 찾기
result = subprocess.run([
    "az", "cognitiveservices", "account", "list",
    "--resource-group", RESOURCE_GROUP,
    "--query", "[?kind=='OpenAI'].{name:name, endpoint:properties.endpoint}",
    "-o", "json"
], capture_output=True, text=True)

openai_accounts = json.loads(result.stdout) if result.stdout else []
print(f"📋 발견된 OpenAI 계정: {len(openai_accounts)}개")

if openai_accounts:
    # 첫 번째 Azure OpenAI 계정 사용
    openai_account = openai_accounts[0]
    openai_name = openai_account['name']
    openai_endpoint = openai_account['endpoint']
    
    print(f"✅ Azure OpenAI 계정: {openai_name}")
    print(f"🔗 Endpoint: {openai_endpoint}")
    
    # 배포된 모델 확인
    result = subprocess.run([
        "az", "cognitiveservices", "account", "deployment", "list",
        "--name", openai_name,
        "--resource-group", RESOURCE_GROUP,
        "--query", "[].{name:name, model:properties.model.name}",
        "-o", "json"
    ], capture_output=True, text=True)
    
    deployments = json.loads(result.stdout) if result.stdout else []
    print(f"\n📦 배포된 모델:")
    for dep in deployments:
        print(f"   - {dep['name']}: {dep['model']}")
    
    # embedding 모델 찾기
    embedding_deployment = None
    for dep in deployments:
        if "embedding" in dep['name'].lower() or "embedding" in dep['model'].lower():
            embedding_deployment = dep['name']
            print(f"\n✅ 임베딩 모델 배포 발견: {embedding_deployment}")
            break
    
    if not embedding_deployment:
        # 기본 이름으로 시도
        embedding_deployment = embedding_model
        print(f"\n💡 기본 배포 이름 사용: {embedding_deployment}")
    
    # REST API URL 구성
    embeddings_url = f"{openai_endpoint.rstrip('/')}/openai/deployments/{embedding_deployment}/embeddings?api-version={api_version}"
    token_scope = "https://cognitiveservices.azure.com/.default"
    
else:
    print("⚠️ Azure OpenAI 계정을 찾을 수 없습니다.")
    print("💡 Foundry 리소스 자체를 사용해봅니다...")
    
    # Foundry 리소스의 엔드포인트 가져오기
    result = subprocess.run([
        "az", "cognitiveservices", "account", "show",
        "--name", FOUNDRY_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--query", "properties.endpoint",
        "-o", "tsv"
    ], capture_output=True, text=True)
    
    foundry_endpoint = result.stdout.strip()
    print(f"🔗 Foundry Endpoint: {foundry_endpoint}")
    
    embeddings_url = f"{foundry_endpoint.rstrip('/')}/openai/deployments/{embedding_model}/embeddings?api-version={api_version}"
    token_scope = "https://cognitiveservices.azure.com/.default"

print(f"\n📍 임베딩 API URL: {embeddings_url}")
print(f"🔐 인증 Scope: {token_scope}")

# Azure 인증 토큰 가져오기
credential = DefaultAzureCredential()

# 배치로 임베딩 생성 (Rate limit 안정적 처리)
batch_size = 30  # Rate limit 고려하여 작은 배치
embedded_chunks = []
max_retries = 5
base_retry_delay = 30  # 기본 대기 시간 30초

print(f"\n⚡ 임베딩 생성 시작 (총 {len(all_chunks)}개, 배치당 {batch_size}개)")
print(f"💡 안정적인 처리를 위해 Rate limit 발생 시 대기합니다.\n")

for i in range(0, len(all_chunks), batch_size):
    batch = all_chunks[i:i+batch_size]
    batch_texts = [chunk['chunk'] for chunk in batch]
    batch_num = i // batch_size + 1
    total_batches = (len(all_chunks) + batch_size - 1) // batch_size
    
    success = False
    for retry in range(max_retries):
        try:
            # 액세스 토큰 가져오기
            token = credential.get_token(token_scope)
            
            # REST API 호출
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {token.token}"
            }
            
            payload = {
                "input": batch_texts,
                "dimensions": 3072
            }
            
            response = requests.post(
                embeddings_url,
                headers=headers,
                json=payload,
                timeout=60
            )
            
            if response.status_code == 200:
                result = response.json()
                
                # 임베딩을 청크에 추가
                for j, chunk in enumerate(batch):
                    chunk['text_vector'] = result['data'][j]['embedding']
                    embedded_chunks.append(chunk)
                
                print(f"  ✅ 배치 {batch_num}/{total_batches}: {len(embedded_chunks)}/{len(all_chunks)} 청크 완료")
                success = True
                break
                
            elif response.status_code == 429:
                # Rate limit 오류 - 지수 백오프로 재시도
                if retry < max_retries - 1:
                    wait_time = base_retry_delay * (2 ** retry)  # 30, 60, 120, 240초...
                    print(f"  ⏳ 배치 {batch_num}/{total_batches}: Rate limit. {wait_time}초 대기 후 재시도 ({retry+1}/{max_retries})...")
                    time.sleep(wait_time)
                else:
                    print(f"  ⚠️ 배치 {batch_num}/{total_batches}: 최대 재시도 횟수 초과 - 스킵")
                    for chunk in batch:
                        chunk['text_vector'] = None
                        embedded_chunks.append(chunk)
                    success = True  # 다음 배치로 진행
                    break
            else:
                print(f"  ⚠️ 배치 {batch_num}/{total_batches} 실패 (HTTP {response.status_code}): {response.text[:150]}")
                for chunk in batch:
                    chunk['text_vector'] = None
                    embedded_chunks.append(chunk)
                success = True  # 다음 배치로 진행
                break
            
        except Exception as e:
            print(f"  ⚠️ 배치 {batch_num}/{total_batches} 오류: {str(e)[:100]}")
            if retry < max_retries - 1:
                print(f"     재시도 {retry + 1}/{max_retries}...")
                time.sleep(5)
            else:
                for chunk in batch:
                    chunk['text_vector'] = None
                    embedded_chunks.append(chunk)
                success = True
                break
    
    # 배치 간 1초 대기 (Rate limit 방지)
    if i + batch_size < len(all_chunks) and success:
        time.sleep(1)

print(f"\n✅ 임베딩 생성 완료: {len(embedded_chunks)}개 청크")
print(f"📊 벡터 차원: {len(embedded_chunks[0]['text_vector']) if embedded_chunks and embedded_chunks[0].get('text_vector') else 0}차원")

### Foundry MI에 Search Index Data Reader 역할 할당

In [ ]:
# Index에 문서 업로드
from azure.core.credentials import AzureKeyCredential

# API 키로 SearchClient 생성 (RBAC 대신 API 키 사용)
upload_credential = AzureKeyCredential(SEARCH_API_KEY) if SEARCH_API_KEY else DefaultAzureCredential()

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=SEARCH_INDEX_NAME,
    credential=upload_credential
)

print("📤 Index에 문서 업로드 중...")
print(f"🔐 인증 방식: {'API 키' if SEARCH_API_KEY else 'DefaultAzureCredential'}\n")

# 배치 업로드 (최대 1000개씩)
batch_size = 30
uploaded_count = 0

for i in range(0, len(embedded_chunks), batch_size):
    batch = embedded_chunks[i:i+batch_size]
    
    # None 벡터 제거 (임베딩 실패한 경우)
    valid_batch = [chunk for chunk in batch if chunk['text_vector'] is not None]
    
    if not valid_batch:
        continue
    
    try:
        # 문서 업로드
        result = search_client.upload_documents(documents=valid_batch)
        
        # 성공한 문서 수 카운트
        succeeded = sum(1 for r in result if r.succeeded)
        uploaded_count += succeeded
        
        print(f"  업로드: {uploaded_count}/{len([c for c in embedded_chunks if c['text_vector']])} 청크")
        
    except Exception as e:
        print(f"  ⚠️ 배치 {i//batch_size + 1} 업로드 실패: {e}")

print(f"\n✅ 인덱싱 완료!")
print(f"   총 업로드: {uploaded_count}개 문서")
print(f"   Index: {SEARCH_INDEX_NAME}")
print(f"\n💡 Index 확인:")
print(f"   https://portal.azure.com → {SEARCH_NAME} → Indexes → {SEARCH_INDEX_NAME}")

### Foundry에 AI Search Connection 생성 (REST API)

## Knowledge Base 생성 (코드 기반)

**중요**: Knowledge Base API는 프리뷰 기능으로, 프리뷰 버전 SDK가 필요합니다.

### Azure Search Documents 프리뷰 버전 설치

In [ ]:
# Knowledge Base API 지원을 위한 프리뷰 버전 설치
print("📦 프리뷰 SDK 설치 중...")
%pip install -q --upgrade azure-search-documents==11.7.0b2

print("\n✅ azure-search-documents 11.7.0b2 설치 완료!")
print("   (Knowledge Base API 지원)")

print("\n⚠️  중요: 커널 재시작 필요!")
print("   1. 상단 메뉴: Kernel > Restart Kernel")
print("   2. 또는 아래 셀 실행하여 자동 재시작")
print("\n💡 커널 재시작 후 다음 셀부터 계속 진행하세요.")

## Foundry Managed Identity에 AI Search 권한 부여 (**필수**)

**⚠️ 중요**: API Key Connection만으로는 **Agent 실행이 불가능**합니다!

**권한 구조:**
- 🔑 **API Key Connection** → 포털에서 Knowledge Base 조회용 (Cell 60)
- 🤖 **Managed Identity 권한** → Agent가 Knowledge Base 실행용 (**이 섹션**)

**왜 필요한가?**
- Agent 실행 시 Foundry의 Managed Identity로 Knowledge Base MCP 엔드포인트에 접근
- API Key는 Connection 설정용이며, Agent 런타임에는 사용되지 않음
- 403 Forbidden 에러 방지를 위해 반드시 설정 필요

In [ ]:
# Foundry Managed Identity에 AI Search 권한 부여 (Agent 실행용 - 필수!)
print("🔐 Foundry Managed Identity에 AI Search 권한 설정 중...")
print("=" * 80)

import json
import uuid
import requests
from azure.identity import DefaultAzureCredential
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import RoleAssignmentCreateParameters

try:
    # config 파일에서 필요한 정보 로드
    config_file = '.foundry_config.json'
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    SEARCH_NAME = config.get("SEARCH_NAME")
    SUBSCRIPTION_ID = config.get("AZURE_SUBSCRIPTION_ID") or config.get("SUBSCRIPTION_ID")
    
    # Foundry 리소스 ID
    foundry_account_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_NAME}"
    
    # AI Search 리소스 ID
    search_resource_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.Search/searchServices/{SEARCH_NAME}"
    
    credential = DefaultAzureCredential()
    
    # Foundry의 Managed Identity Principal ID 가져오기
    print(f"\n🔍 Foundry Managed Identity 확인 중...")
    
    token = credential.get_token("https://management.azure.com/.default")
    headers = {
        "Authorization": f"Bearer {token.token}",
        "Content-Type": "application/json"
    }
    
    foundry_url = f"https://management.azure.com{foundry_account_id}?api-version=2025-09-01"
    response = requests.get(foundry_url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Foundry 정보 조회 실패: {response.status_code}")
    
    foundry_data = response.json()
    principal_id = foundry_data.get("identity", {}).get("principalId")
    
    if not principal_id:
        print(f"   ⚠️ Foundry의 Managed Identity가 활성화되지 않았습니다.")
        print(f"   💡 Managed Identity 활성화 중...")
        
        # Managed Identity 활성화
        foundry_data["identity"] = {"type": "SystemAssigned"}
        update_response = requests.patch(foundry_url, headers=headers, json={"identity": {"type": "SystemAssigned"}})
        
        if update_response.status_code in [200, 201]:
            principal_id = update_response.json().get("identity", {}).get("principalId")
            print(f"   ✅ Managed Identity 활성화 완료!")
        else:
            raise Exception(f"Managed Identity 활성화 실패: {update_response.text}")
    
    print(f"   ✅ Principal ID: {principal_id}")
    
    # Role Assignment 생성
    auth_client = AuthorizationManagementClient(credential, SUBSCRIPTION_ID)
    
    # 필요한 역할 정의 (Agent 실행용)
    roles_to_assign = [
        {
            "name": "Search Index Data Reader",
            "id": "1407120a-92aa-4202-b7e9-c0e197c71c8f",
            "purpose": "Knowledge Base 데이터 읽기"
        },
        {
            "name": "Search Service Contributor",
            "id": "7ca78c08-252a-4471-8644-bb5ff32d4ba0",
            "purpose": "Knowledge Base MCP 엔드포인트 접근"
        }
    ]
    
    for role in roles_to_assign:
        print(f"\n🚀 '{role['name']}' 권한 확인 중...")
        print(f"   목적: {role['purpose']}")
        
        role_definition_id = f"/subscriptions/{SUBSCRIPTION_ID}/providers/Microsoft.Authorization/roleDefinitions/{role['id']}"
        
        # 기존 role assignment 확인
        existing_assignments = list(auth_client.role_assignments.list_for_scope(
            scope=search_resource_id,
            filter=f"principalId eq '{principal_id}'"
        ))
        
        has_role = any(
            role['id'] in assignment.role_definition_id
            for assignment in existing_assignments
        )
        
        if has_role:
            print(f"   ✅ '{role['name']}' 권한이 이미 존재합니다.")
        else:
            # 새 Role Assignment 생성
            role_assignment_name = str(uuid.uuid4())
            role_assignment_params = RoleAssignmentCreateParameters(
                role_definition_id=role_definition_id,
                principal_id=principal_id,
                principal_type="ServicePrincipal"
            )
            
            assignment = auth_client.role_assignments.create(
                scope=search_resource_id,
                role_assignment_name=role_assignment_name,
                parameters=role_assignment_params
            )
            
            print(f"   ✅ '{role['name']}' 권한 부여 완료!")
            print(f"   Assignment ID: {assignment.name}")
    
    print(f"\n" + "=" * 80)
    print(f"✅ Agent 실행용 권한 설정 완료!")
    print(f"\n⚠️ 중요: 권한이 전파되는데 2-3분 소요됩니다.")
    print(f"💡 2-3분 후:")
    print(f"   1. Agent 테스트 실행 가능")
    print(f"   2. 403 Forbidden 에러 해결")
    print(f"   3. Knowledge Base MCP 엔드포인트 정상 접근")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ 권한 설정 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()


### Azure AI Search 연결 추가

Foundry IQ에서 Knowledge Base를 사용하려면 먼저 AI Search 리소스를 연결해야 합니다.

**Foundry IQ에서 AI Search 연결하기:**

1. **Azure AI Foundry Portal 접속**
   - https://ai.azure.com 접속
   - 생성한 Foundry 프로젝트 선택

2. **Foundry IQ 메뉴로 이동**
   - 왼쪽 메뉴에서 **Foundry IQ** 클릭
   - "Ground your agent in enterprise knowledge" 화면이 표시됨

3. **AI Search 리소스 연결**
   - **Azure AI Search resource** 드롭다운 클릭
   - 생성한 AI Search 서비스 선택
   - **Connect** 버튼 클릭

4. **연결 완료**
   - AI Search 리소스가 성공적으로 연결되면 Knowledge bases와 Indexes 탭이 활성화됩니다

> **💡 Tip**: 
> - AI Search 리소스가 목록에 없다면 "Create new resource" 링크를 클릭하여 새로 생성할 수 있습니다
> - 연결 후에는 코드에서 `project_client.connections.list()`로 연결을 확인할 수 있습니다

## Knowledge Base 생성

In [ ]:
# Knowledge Base를 위한 라이브러리 import
import json
import subprocess
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    KnowledgeBase,
    KnowledgeSourceReference,
    KnowledgeBaseAzureOpenAIModel,
    SearchIndexKnowledgeSource,
    SearchIndexKnowledgeSourceParameters,
    SearchIndexFieldReference,
    AzureOpenAIVectorizerParameters,
    KnowledgeRetrievalOutputMode
)

# config 파일 로드
config_file = '.foundry_config.json'
with open(config_file, 'r') as f:
    config = json.load(f)

SEARCH_NAME = config.get("SEARCH_NAME")

# AI Search API 키 획득 (명령어로 조회)
result = subprocess.run([
    "az", "search", "admin-key", "show",
    "--resource-group", RESOURCE_GROUP,
    "--service-name", SEARCH_NAME,
    "--query", "primaryKey", "-o", "tsv"
], capture_output=True, text=True)

SEARCH_API_KEY = result.stdout.strip()
search_endpoint = f"https://{SEARCH_NAME}.search.windows.net"

# SearchIndexClient 초기화 (API Key 사용)
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(SEARCH_API_KEY)
)

print("✅ Knowledge Base 라이브러리 import 완료")
print(f"   - KnowledgeBase")
print(f"   - KnowledgeSourceReference")
print(f"   - KnowledgeBaseAzureOpenAIModel")
print(f"   - SearchIndexKnowledgeSource")
print(f"   - SearchIndexKnowledgeSourceParameters")
print(f"   - SearchIndexFieldReference")
print(f"✅ SearchIndexClient 초기화 완료: {search_endpoint}")


### Knowledge Source 생성 (Search Index 연결)

In [ ]:
print("🔗 1단계: Knowledge Source 생성 중...")
KNOWLEDGE_SOURCE_NAME = "knowledge-source-01"

try:
    knowledge_source = SearchIndexKnowledgeSource(
        name=KNOWLEDGE_SOURCE_NAME,
        description="Foundry IQ Knowledge Source from existing search index",
        search_index_parameters=SearchIndexKnowledgeSourceParameters(
            search_index_name=SEARCH_INDEX_NAME,
            # 검색에 사용할 필드들 지정 (실제 인덱스 필드 이름 사용)
            source_data_fields=[
                SearchIndexFieldReference(name="chunk"),  # 텍스트 콘텐츠
                SearchIndexFieldReference(name="title")   # 문서 제목
            ],
            # 검색 필드 (키 필드 - 실제 인덱스의 키 필드 이름 사용)
            search_fields=[
                SearchIndexFieldReference(name="chunk_id")
            ]
        )
    )
    
    result_ks = index_client.create_or_update_knowledge_source(knowledge_source)
    print(f"   ✅ Knowledge Source 생성 완료: {KNOWLEDGE_SOURCE_NAME}")
except Exception as e:
    print(f"   ⚠️ Knowledge Source 생성 실패: {e}")
    print("   💡 이미 존재하는 경우 무시하고 계속 진행합니다.")

### Knowledge Base 생성

In [ ]:
print("📚 2단계: Knowledge Base 생성 중...")
KNOWLEDGE_BASE_NAME = "foundry-knowledge-base"

# config에서 필요한 변수 로드
FOUNDRY_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
SEARCH_INDEX_NAME = config.get("SEARCH_INDEX_NAME", "knowledge-index")
KNOWLEDGE_SOURCE_NAME = config.get("KNOWLEDGE_SOURCE_NAME", "knowledge-source-01")

# Azure OpenAI 파라미터 설정
# answer synthesis를 위한 GPT 모델 사용
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=FOUNDRY_ENDPOINT,  # Foundry 엔드포인트 사용
    deployment_name="gpt-4.1",  # GPT 배포 이름
    model_name="gpt-4.1"
)

try:
    knowledge_base = KnowledgeBase(
        name=KNOWLEDGE_BASE_NAME,
        description="Foundry IQ Knowledge Base for HR documents",
        retrieval_instructions="이 지식 기반은 HR 관련 문서를 포함하고 있습니다. 직원 복리후생, 채용 정책 등에 대한 질문에 답변하세요.",
        answer_instructions="검색된 문서를 기반으로 명확하고 간결한 답변을 제공하세요. 문서에서 직접 인용하여 신뢰성을 높이세요.",
        output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,  # 자동 답변 합성
        knowledge_sources=[
            KnowledgeSourceReference(name=KNOWLEDGE_SOURCE_NAME)
        ],
        models=[
            KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)
        ]
    )
    
    result_kb = index_client.create_or_update_knowledge_base(knowledge_base)
    print(f"   ✅ Knowledge Base 생성 완료!")
    print(f"\n📋 Knowledge Base 정보:")
    print(f"   이름: {KNOWLEDGE_BASE_NAME}")
    print(f"   Knowledge Source: {KNOWLEDGE_SOURCE_NAME}")
    print(f"   Search Index: {SEARCH_INDEX_NAME}")
    print(f"   Output Mode: Answer Synthesis")
    
    # config에 저장
    config["KNOWLEDGE_BASE_NAME"] = KNOWLEDGE_BASE_NAME
    config["KNOWLEDGE_SOURCE_NAME"] = KNOWLEDGE_SOURCE_NAME
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n💡 Knowledge Base 확인:")
    print(f"   https://portal.azure.com → {SEARCH_NAME} → Knowledge bases")
    
except Exception as e:
    import traceback
    print(f"   ⚠️ Knowledge Base 생성 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print("\n💡 문제 해결:")
    print("   1. 커널을 재시작했는지 확인")
    print("   2. azure-search-documents 프리뷰 버전(11.7.0b2)이 설치되었는지 확인")
    print("   3. Azure OpenAI 모델 'gpt-4.1'가 배포되어 있는지 확인")
    print("   4. Search API Key 권한이 충분한지 확인")

### KnowledgeAgent 생성

In [ ]:
# Connection에 API Key 설정 (403 Forbidden 해결)
print("🔑 AI Search Connection API Key 설정 중...")
print("=" * 80)

import json
import subprocess
import requests
from azure.identity import DefaultAzureCredential

try:
    # config 파일에서 필요한 정보 로드
    config_file = '.foundry_config.json'
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    SEARCH_NAME = config.get("SEARCH_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    SUBSCRIPTION_ID = config.get("AZURE_SUBSCRIPTION_ID") or config.get("SUBSCRIPTION_ID")
    PROJECT_ENDPOINT = config.get("PROJECT_ENDPOINT") or config.get("FOUNDRY_ENDPOINT")
    
    # AI Search API 키 획득
    print(f"\n🔍 AI Search API Key 조회 중...")
    result = subprocess.run([
        "az", "search", "admin-key", "show",
        "--resource-group", RESOURCE_GROUP,
        "--service-name", SEARCH_NAME,
        "--query", "primaryKey", "-o", "tsv"
    ], capture_output=True, text=True)
    
    if result.returncode != 0:
        raise Exception(f"API Key 조회 실패: {result.stderr}")
    
    api_key = result.stdout.strip()
    print(f"   ✅ API Key 획득 완료")
    
    # Project Connection ID 확인
    from azure.ai.projects import AIProjectClient
    credential = DefaultAzureCredential()
    project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
    
    connections = project_client.connections.list()
    search_connections = [c for c in connections if 'search' in c.name.lower()]
    
    if not search_connections:
        raise ValueError("AI Search Connection을 찾을 수 없습니다.")
    
    connection = search_connections[0]
    CONNECTION_NAME = connection.name
    
    print(f"\n🔧 Connection 업데이트 중: {CONNECTION_NAME}")
    
    # Connection 업데이트 (REST API 사용)
    token = credential.get_token("https://management.azure.com/.default")
    headers = {
        "Authorization": f"Bearer {token.token}",
        "Content-Type": "application/json"
    }
    
    # Project 리소스 ID 추출
    project_id = PROJECT_ENDPOINT.split("/projects/")[0].replace("https://", "").replace(".api.azureml.ms", "")
    project_name = PROJECT_ENDPOINT.split("/projects/")[1]
    
    # Foundry 계정 정보 확인
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    foundry_base_url = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_NAME}"
    
    # Connection URL
    connection_url = f"{foundry_base_url}/projects/{project_name}/connections/{CONNECTION_NAME}?api-version=2025-09-01"
    
    # Connection 정보 조회
    response = requests.get(connection_url, headers=headers)
    if response.status_code == 200:
        connection_data = response.json()
        
        # API Key로 업데이트
        connection_data["properties"]["credentials"] = {
            "type": "ApiKey",
            "key": api_key
        }
        
        # Connection 업데이트
        update_response = requests.put(connection_url, headers=headers, json=connection_data)
        
        if update_response.status_code in [200, 201]:
            print(f"   ✅ Connection API Key 설정 완료!")
            print(f"\n💡 이제 Agent가 Knowledge Base에 접근할 수 있습니다.")
        else:
            print(f"   ⚠️ Connection 업데이트 실패: {update_response.status_code}")
            print(f"   응답: {update_response.text}")
    else:
        print(f"   ⚠️ Connection 조회 실패: {response.status_code}")
        print(f"   응답: {response.text}")
        
        # 대안: Portal에서 수동 설정 안내
        print(f"\n📝 수동 설정 방법:")
        print(f"   1. https://ai.azure.com 접속")
        print(f"   2. Build → Connections → {CONNECTION_NAME} 선택")
        print(f"   3. 'Edit' 클릭")
        print(f"   4. Authentication 섹션에서 'API Key' 선택")
        print(f"   5. API Key 입력 후 저장")
    
    print(f"\n" + "=" * 80)
    print(f"✅ Connection 설정 완료!")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ Connection 설정 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print(f"\n💡 수동 설정:")
    print(f"   Portal에서 Connection의 API Key를 직접 설정하세요.")


### Connection 생성 (API Key 인증)

Agent가 Knowledge Base에 접근하려면 AI Search Connection이 API Key로 인증되어야 합니다.

In [ ]:
print("🤖 3단계: KnowledgeAgent 생성 중...")
print("=" * 80)

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool
from azure.identity import DefaultAzureCredential

try:
    # config 파일에서 필요한 정보 로드
    config_file = '.foundry_config.json'
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    PROJECT_ENDPOINT = config.get("PROJECT_ENDPOINT") or config.get("FOUNDRY_ENDPOINT")
    SEARCH_NAME = config["SEARCH_NAME"]
    KNOWLEDGE_BASE_NAME = config.get("KNOWLEDGE_BASE_NAME", "foundry-knowledge-base")
    
    # AIProjectClient 생성 (Connection 조회용)
    credential = DefaultAzureCredential()
    project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
    
    # Connection 이름 확인 (실제 존재하는 Connection 사용)
    print(f"\n🔍 프로젝트 Connection 목록 확인 중...")
    connections = project_client.connections.list()
    search_connections = [c for c in connections if 'search' in c.name.lower()]
    
    if not search_connections:
        raise ValueError(f"❌ AI Search Connection을 찾을 수 없습니다.\n"
                        f"💡 Portal에서 AI Search Connection을 먼저 생성하세요:\n"
                        f"   https://ai.azure.com → Build → Connections → + Connection → Azure AI Search")
    
    # 첫 번째 Search Connection 사용
    PROJECT_CONNECTION_NAME = search_connections[0].name
    print(f"   ✅ Connection 발견: {PROJECT_CONNECTION_NAME}")
    
    # MCP 엔드포인트 URL (api-version 포함, API Key는 Connection을 통해 전달)
    MCP_ENDPOINT = f"https://{SEARCH_NAME}.search.windows.net/knowledgebases/{KNOWLEDGE_BASE_NAME}/mcp?api-version=2025-11-01-preview"
    
    print(f"\n📋 Agent 설정:")
    print(f"   Project Endpoint: {PROJECT_ENDPOINT}")
    print(f"   Connection: {PROJECT_CONNECTION_NAME}")
    print(f"   MCP Endpoint: {MCP_ENDPOINT}")
    print(f"\n💡 인증 방식: Connection을 통한 API Key 자동 전달")
    
    # Agent Instructions (Microsoft 권장 템플릿)
    KNOWLEDGE_INSTRUCTIONS = """You are a helpful assistant that must use the knowledge base to answer all the questions from user. You must never answer from your own knowledge under any circumstances.

Every answer must always provide annotations for using the MCP knowledge base tool and render them as: 【message_idx:search_idx†source_name】

If you cannot find the answer in the provided knowledge base you must respond with "I don't know".

한국어로 답변하세요."""
    
    # MCP Tool 설정 (Connection을 통해 API Key 전달)
    print(f"\n🔧 MCP Tool 구성 중...")
    print(f"   Server Label: knowledge-base")
    print(f"   Connection ID: {PROJECT_CONNECTION_NAME} (API Key 인증)")
    
    mcp_kb_tool = MCPTool(
        server_label="knowledge-base",
        server_url=MCP_ENDPOINT,
        require_approval="never",
        allowed_tools=["knowledge_base_retrieve"],
        project_connection_id=PROJECT_CONNECTION_NAME
    )
    
    print(f"\n🚀 Agent 생성 중...")
    
    # Agent 생성
    agent = project_client.agents.create_version(
        agent_name="KnowledgeAgent",
        definition=PromptAgentDefinition(
            model="gpt-5.1",
            instructions=KNOWLEDGE_INSTRUCTIONS,
            tools=[mcp_kb_tool]
        )
    )
    
    print(f"   ✅ KnowledgeAgent 생성 완료!")
    print(f"\n📋 Agent 정보:")
    print(f"   이름: {agent.name}")
    print(f"   버전: {agent.version}")
    print(f"   모델: gpt-5.1")
    print(f"   MCP Tool: knowledge_base_retrieve")
    
    # config에 저장
    config["AGENT_NAME"] = agent.name
    config["AGENT_VERSION"] = agent.version
    config["PROJECT_CONNECTION_NAME"] = PROJECT_CONNECTION_NAME  # 실제 Connection 이름 저장
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n💡 Agent 확인:")
    print(f"   https://ai.azure.com → Build → Agents → {agent.name}")
    
except Exception as e:
    import traceback
    print(f"   ⚠️ Agent 생성 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print("\n💡 문제 해결:")
    print("   1. Project Connection이 먼저 생성되었는지 확인")
    print("   2. Managed Identity 권한이 설정되었는지 확인")
    print("   3. Azure CLI 로그인 상태 확인")
    print("   4. gpt-5.1 모델이 배포되어 있는지 확인")

### Agent 테스트

> **⚠️ 실행 전 필수 작업**: [Azure AI Foundry Portal](https://ai.azure.com)에서 `foundry-knowledge-base` Knowledge Base를 Project에 연결해야 합니다.

In [ ]:
# KnowledgeAgent 테스트
print("🧪 4단계: KnowledgeAgent 테스트 시작...")
print("=" * 80)

try:
    # Agent 변수 검증
    if 'agent' in locals():
        knowledge_agent = agent
        print(f"✅ Agent 변수 사용: {knowledge_agent.name} (version: {knowledge_agent.version})")
    elif 'knowledge_agent' not in locals():
        raise NameError("knowledge_agent가 정의되지 않았습니다. 위의 KnowledgeAgent 생성 셀을 먼저 실행하세요.")
    
    # OpenAI Client 검증
    if 'openai_client' not in locals():
        raise NameError("OpenAI client가 없습니다. Setup 셀을 먼저 실행하세요.")
    
    # Test Questions (첫 번째만 테스트)
    test_questions = [
        "PerkPlus가 커버하는 항목들을 알려줘"
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[질문 {i}]: {question}")
        print("-" * 80)
        
        # Conversation 생성
        conversation = openai_client.conversations.create()
        
        # Agent를 통해 응답 생성
        print(f"  응답 생성 중...", flush=True)
        response = openai_client.responses.create(
            conversation=conversation.id,
            input=question,
            extra_body={"agent": {"name": knowledge_agent.name, "type": "agent_reference"}}
        )
        
        # DEBUG: Response 상태 확인
        print(f"\n🔍 Response 상태: {response.status}")
        print(f"🔍 Response ID: {response.id}")
        
        if hasattr(response, 'output') and isinstance(response.output, list):
            print(f"🔍 Output 길이: {len(response.output)}")
            for idx, item in enumerate(response.output):
                print(f"\n  Item {idx}: {item.__class__.__name__}")
                if hasattr(item, 'type'):
                    print(f"    type: {item.type}")
                if hasattr(item, 'id'):
                    print(f"    id: {item.id}")
                if hasattr(item, 'text'):
                    print(f"    text (처음 100자): {item.text[:100]}")
                if item.__class__.__name__ == 'McpApprovalRequest':
                    print(f"    ⚠️ Approval 필요!")
                    print(f"    name: {item.name if hasattr(item, 'name') else 'N/A'}")
                    print(f"    arguments: {item.arguments[:200] if hasattr(item, 'arguments') else 'N/A'}")
        
        # 응답 텍스트 추출
        actual_text = ""
        
        if hasattr(response, 'output') and isinstance(response.output, list):
            for item in response.output:
                if hasattr(item, 'text'):
                    actual_text = item.text
                    break
        
        print(f"\n[응답]: {actual_text if actual_text else '⚠️ 응답을 찾을 수 없습니다'}\n")
    
    print("=" * 80)
    print("✅ KnowledgeAgent 테스트 완료!")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ 테스트 실패: {e}")
    traceback.print_exc()

## Azure Blob Storage 기반 Knowledge Base

Blob Storage를 직접 연결하여 더 간단하게 Knowledge Base를 생성합니다.

### 1. Storage Account 정보 확인

In [ ]:
# config 파일에서 Storage Account 이름 로드
try:
    STORAGE_NAME = config.get("STORAGE_NAME")
    if not STORAGE_NAME:
        raise ValueError("STORAGE_NAME이 config에 없습니다.")
except:
    print("⚠️ config에서 STORAGE_NAME을 찾을 수 없습니다.")
    print("💡 위의 '환경 변수 로드' 셀과 'AI Search 및 Storage 리소스 이름 생성' 셀을 먼저 실행하세요.")
    raise

# Container 이름 설정
CONTAINER_NAME = "documents"

print(f"Storage Account: {STORAGE_NAME}")
print(f"Container: {CONTAINER_NAME}")
print(f"\n✅ 이미 생성된 Storage Account와 Container를 사용합니다.")
print(f"💡 동일한 Blob Storage를 사용하므로 추가 설정 불필요")

### 2. Knowledge Source 생성 (Blob Storage)

In [ ]:
import subprocess
from azure.search.documents.indexes.models import (
    AzureBlobKnowledgeSource,
    AzureBlobKnowledgeSourceParameters,
    KnowledgeSourceIngestionParameters,
    KnowledgeSourceContentExtractionMode,
    KnowledgeSourceAzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)

# Knowledge Source 이름
BLOB_KNOWLEDGE_SOURCE_NAME = "ks-azureblob-200"

print("Knowledge Source 생성 중...")
print(f"이름: {BLOB_KNOWLEDGE_SOURCE_NAME}")

try:
    # Storage Account Resource ID 구성
    storage_resource_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.Storage/storageAccounts/{STORAGE_NAME}"
    
    # Embedding Model (Vectorizer) 설정
    aoai_vectorizer_params = AzureOpenAIVectorizerParameters(
        resource_url=FOUNDRY_ENDPOINT,
        deployment_name="text-embedding-3-large",
        model_name="text-embedding-3-large"
    )
    
    embedding_vectorizer = KnowledgeSourceAzureOpenAIVectorizer(
        azure_open_ai_parameters=aoai_vectorizer_params
    )
    
    # Ingestion 파라미터 설정 (System-Assigned MI는 identity 파라미터 불필요)
    ingestion_params = KnowledgeSourceIngestionParameters(
        content_extraction_mode=KnowledgeSourceContentExtractionMode.MINIMAL,
        embedding_model=embedding_vectorizer
    )
    
    # Blob Storage 파라미터 설정 (Managed Identity Connection String 사용)
    # System-Assigned Managed Identity를 사용할 때는 ResourceId 형식의 connection string만 필요
    managed_identity_connection_string = f"ResourceId={storage_resource_id};"
    
    blob_params = AzureBlobKnowledgeSourceParameters(
        connection_string=managed_identity_connection_string,
        container_name=CONTAINER_NAME,
        ingestion_parameters=ingestion_params
    )
    
    # Blob Storage 기반 Knowledge Source 생성
    blob_knowledge_source = AzureBlobKnowledgeSource(
        name=BLOB_KNOWLEDGE_SOURCE_NAME,
        azure_blob_parameters=blob_params,
        description="Blob Storage 기반 Knowledge Source"
    )
    
    # Knowledge Source 생성
    result_blob_ks = index_client.create_knowledge_source(blob_knowledge_source)
    
    print(f"\n✅ Knowledge Source 생성 완료!")
    print(f"   - 이름: {result_blob_ks.name}")
    print(f"   - 타입: Blob Storage")
    print(f"   - Container: {CONTAINER_NAME}")
    print(f"   - Embedding 모델: text-embedding-3-large")
    print(f"\n💡 이 Knowledge Source는 Blob Storage의 파일을 자동으로 인덱싱합니다.")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ Knowledge Source 생성 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print("\n💡 해결 방법:")
    print("   1. Storage Account와 Container가 올바른지 확인")
    print("   2. AI Search Managed Identity가 Storage에 대한 권한이 있는지 확인")
    print("   3. FOUNDRY_ENDPOINT가 올바르게 설정되었는지 확인")
    print("   4. 이미 존재하는 경우 다른 이름 사용")

### 3. Knowledge Base 생성 (Blob Storage 기반)

> **⚠️ 실행 전 필수 작업**: [Azure AI Foundry Portal](https://ai.azure.com)에서 `blob-knowledge-base` Knowledge Base를 Project에 연결해야 합니다.

In [ ]:
from azure.search.documents.indexes.models import (
    KnowledgeSourceReference,
    KnowledgeBaseAzureOpenAIModel
)

# Knowledge Base 이름
BLOB_KNOWLEDGE_BASE_NAME = "knowledgebase200"

print("Knowledge Base 생성 중...")
print(f"이름: {BLOB_KNOWLEDGE_BASE_NAME}")

try:
    # Azure OpenAI 모델 설정 (Answer Synthesis용)
    aoai_model_params = AzureOpenAIVectorizerParameters(
        resource_url=FOUNDRY_ENDPOINT,
        deployment_name="gpt-5.1",
        model_name="gpt-5.1"
    )
    
    # Knowledge Base 생성 (Blob Storage 기반)
    blob_knowledge_base = KnowledgeBase(
        name=BLOB_KNOWLEDGE_BASE_NAME,
        description="Blob Storage 기반 Knowledge Base",
        retrieval_instructions="이 지식 기반은 Blob Storage의 문서를 포함하고 있습니다.",
        answer_instructions="검색된 문서를 기반으로 명확한 답변을 제공하세요.",
        output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
        knowledge_sources=[
            KnowledgeSourceReference(name=BLOB_KNOWLEDGE_SOURCE_NAME)
        ],
        models=[
            KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_model_params)
        ]
    )
    
    # Knowledge Base 생성
    result_blob_kb = index_client.create_or_update_knowledge_base(blob_knowledge_base)
    
    print(f"\n✅ Knowledge Base 생성 완료!")
    print(f"   - 이름: {result_blob_kb.name}")
    print(f"   - Output Mode: {result_blob_kb.output_mode}")
    print(f"   - Knowledge Source: {BLOB_KNOWLEDGE_SOURCE_NAME}")
    print(f"   - 모델: gpt-5.1")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ Knowledge Base 생성 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print("\n💡 해결 방법:")
    print("   1. Knowledge Source가 먼저 생성되었는지 확인")
    print("   2. 이미 존재하는 경우 다른 이름 사용")
    print("   3. Chat 모델 이름 확인")

### 4. KnowledgeAgent2 생성 (Blob Storage 기반)

In [ ]:
KNOWLEDGE_AGENT_2_NAME = "KnowledgeAgent2"

print("KnowledgeAgent2 생성 중...")
print(f"Knowledge Base: {BLOB_KNOWLEDGE_BASE_NAME}")

try:
    # MCP Tool 정의 (Blob Storage 기반 Knowledge Base)
    mcp_blob_kb_tool = MCPTool(
        server_label="knowledge-base-blob",
        server_url=MCP_ENDPOINT,
        require_approval="never",
        allowed_tools=["knowledge_base_retrieve"],
        project_connection_id=PROJECT_CONNECTION_NAME
    )
    
    # Agent 생성
    blob_knowledge_agent = project_client.agents.create_version(
        agent_name=KNOWLEDGE_AGENT_2_NAME,
        definition=PromptAgentDefinition(
            model="model-router",
            instructions=KNOWLEDGE_INSTRUCTIONS,
            tools=[mcp_blob_kb_tool]
        )
    )
    
    print(f"\n✅ {KNOWLEDGE_AGENT_2_NAME} 생성 완료!")
    print(f"   - Agent: {KNOWLEDGE_AGENT_2_NAME}")
    print(f"   - Model: model-router")
    print(f"   - Knowledge Base: {BLOB_KNOWLEDGE_BASE_NAME}")
    print(f"   - 데이터 소스: Blob Storage")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ Agent 생성 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()
    print("\n💡 해결 방법:")
    print("   1. Knowledge Base가 먼저 생성되었는지 확인")
    print("   2. MCP endpoint 확인")
    print("   3. PROJECT_CONNECTION_NAME이 올바른지 확인")

### 5. KnowledgeAgent2 테스트

In [ ]:
import time
import json

print(f"{'=' * 80}")
print(f"KnowledgeAgent2 테스트 (Blob Storage 기반)")
print(f"{'=' * 80}\n")

try:
    # OpenAI client 가져오기
    openai_client = project_client.get_openai_client()
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[질문 {i}]: {question}")
        print("-" * 80)
        
        # Conversation 생성
        conversation = openai_client.conversations.create()
        
        # Agent를 통해 응답 생성 (Rate limit 재시도 로직 포함)
        print(f"  응답 생성 중...", flush=True)
        
        max_retries = 3
        retry_delay = 15  # 초
        response = None
        
        for attempt in range(max_retries):
            try:
                response = openai_client.responses.create(
                    conversation=conversation.id,
                    input=question,
                    extra_body={"agent": {"name": blob_knowledge_agent.name, "type": "agent_reference"}}
                )
                print(f"  ✅ 응답 생성 성공!")
                break  # 성공하면 루프 종료
                
            except Exception as e:
                error_message = str(e)
                if "429" in error_message or "Too Many Requests" in error_message:
                    if attempt < max_retries - 1:
                        wait_time = retry_delay * (attempt + 1)
                        print(f"  ⏳ Rate limit 도달. {wait_time}초 대기 후 재시도... ({attempt + 1}/{max_retries})")
                        time.sleep(wait_time)
                    else:
                        print(f"  ❌ {max_retries}번 재시도 후에도 실패.")
                        print(f"  💡 5분 정도 기다린 후 다시 시도하세요.")
                        raise
                else:
                    # 다른 에러는 바로 throw
                    raise
        
        if response is None:
            print("⚠️ 응답을 받지 못했습니다.")
            continue
        
        # DEBUG: Response 상태 확인
        print(f"\n🔍 Response 상태: {response.status}")
        print(f"🔍 Response ID: {response.id}")
        
        if hasattr(response, 'output') and isinstance(response.output, list):
            print(f"🔍 Output 길이: {len(response.output)}")
            for idx, item in enumerate(response.output):
                print(f"\n  Item {idx}: {item.__class__.__name__}")
                if hasattr(item, 'type'):
                    print(f"    type: {item.type}")
                if hasattr(item, 'id'):
                    print(f"    id: {item.id}")
                if hasattr(item, 'text'):
                    print(f"    text (처음 100자): {item.text[:100]}")
                if item.__class__.__name__ == 'McpApprovalRequest':
                    print(f"    ⚠️ Approval 필요!")
                    print(f"    name: {item.name if hasattr(item, 'name') else 'N/A'}")
                    print(f"    arguments: {item.arguments[:200] if hasattr(item, 'arguments') else 'N/A'}")
        
        # 응답 텍스트 추출
        actual_text = ""
        
        if hasattr(response, 'output') and isinstance(response.output, list):
            for item in response.output:
                if hasattr(item, 'text'):
                    actual_text = item.text
                    break
        
        print(f"\n[응답]: {actual_text if actual_text else '⚠️ 응답을 찾을 수 없습니다'}\n")
    
    print("=" * 80)
    print("✅ KnowledgeAgent2 테스트 완료!")
    
except Exception as e:
    import traceback
    print(f"\n⚠️ 테스트 실패: {e}")
    print("\n상세 오류:")
    traceback.print_exc()


### 📝 두 방식 비교

| 특징 | AI Search Index | Blob Storage 직접 연결 |
|------|----------------|----------------------|
| **설정 복잡도** | 높음 (Import Wizard 필요) | 낮음 (간단한 설정) |
| **자동 업데이트** | 수동 재인덱싱 필요 | 자동 감지 및 인덱싱 |
| **커스터마이징** | 높음 (필드, 스키마 등) | 낮음 (자동 구성) |
| **성능** | 높음 (최적화 가능) | 중간 |
| **사용 사례** | 복잡한 검색 요구사항 | 간단한 문서 검색 |

**권장 사항:**
- 복잡한 검색 및 필터링이 필요한 경우 → AI Search Index
- 빠른 프로토타입 및 간단한 문서 검색 → Blob Storage

## 📚 추가 리소스

- [Foundry IQ 개요](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/knowledge-retrieval?view=foundry&tabs=foundry%2Cpython)
- [Azure AI Search 문서](https://learn.microsoft.com/en-us/azure/search/)
- [RAG 패턴 가이드](https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview?tabs=docs)
- [벡터 검색 최적화](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)